In [ ]:
! pip install transformers
! pip install datasets
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score


In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 17.9 MB/s 


In [3]:

from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time
from torchmetrics import MeanSquaredError
import torch

In [4]:

import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/新建文件夹/ERAI Dataset_with_translation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/新建文件夹/ERAI Dataset_with_translation


In [6]:
pd.set_option('display.max_rows', None)



In [ ]:

df_unsupervised = pd.read_json('/content/drive/MyDrive/Colab Notebooks/新建文件夹/ERAI_Dataset_unsupervised.json')

In [9]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
# df2 = pd.read_json('ERAI_Dataset_pairwise_test.json')

from sklearn.utils import shuffle
df = shuffle(df)
# df2 = shuffle(df2)


df['post1_en']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2'].apply(lambda x:x.replace('\n',''))
# df2['post1']=df2['post1'].apply(lambda x:x.replace('\n',''))
# df2['post2']=df2['post2'].apply(lambda x:x.replace('\n',''))

In [10]:
df = df.dropna(axis = 0)

In [ ]:
# df['MPP1'] = df['MPP1'].apply(lambda x:x*10)
# df['MPP2'] = df['MPP2'].apply(lambda x:x*10)
# df['ML1'] = df['ML1'].apply(lambda x:x*10)
# df['ML2'] = df['ML2'].apply(lambda x:x*10)

In [12]:
PRETRAINED_MODEL_NAME = "nlpaueb/sec-bert-shape"  
# 获取预测模型所使用的tokenizer
# AutoTokenizer, AutoModel,
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [13]:

def generate_input_data(post,mpp_all_value,ml_all_value,max_len):

  # mpp_all_value = np.array(mpp_all_value)
  # ml_all_value = np.arrary(ml_all_value)
  # all_value = mpp_all_value+ml_all_value
  all_value = pd.concat([mpp_all_value,ml_all_value],axis=1)
  
  
  full_text = []
  attention_mask = []
  # token_type_ids = []
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    # token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    # token_type_ids.append(token_type_id)
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  # token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      # 'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'all_value': torch.tensor(all_value.values, dtype=torch.float)
      # 'ml_label':torch.tensor(ml_label,dtype = torch.long)
  }

    

In [14]:
 class BertNerClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("nlpaueb/sec-bert-shape")
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask

    )[1]
    
    output = self.drop(output)
    # self.out(output)
    
    return self.out(output)

In [15]:

class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      # self.token_ids = data['token_type_ids']
      self.all_value = data['all_value']
      # self.ml_labels = data['ml_label']

    def __len__(self):

        return len(self.all_value)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.all_value[index]

In [16]:

# test_post2 = post2[180:200]

# train_mpp_label = mpp_label[:160]
# validation_mpp_label = mpp_label[160:180]
# test_mpp_label = mpp_label[180:200]

# train_ml_label = ml_label[:160]
# validation_ml_label = ml_label[160:180]
# test_ml_label = ml_label[180:200]
# train_mpp_label = mpp_label[:160]
train = df[0:180]
# validation = df[160:180]
test = df[176:200]
train_post1 = train['post1_en']
train_post2 = train['post2_en']
train_mmp1 = train['MPP1']
train_mpp2 = train['MPP2']
train_ml1 = train['ML1']
train_ml2 = train['ML2']


In [17]:
all_train_post = pd.concat([train_post1,train_post2],axis=0)
all_train_mpp = pd.concat([train_mmp1,train_mpp2],axis=0)
all_train_ml = pd.concat([train_ml1,train_ml2],axis=0)

In [ ]:
# validation_post1 = validation['post1']
# validation_post2 = validation['post2']
# validation_mmp1 = validation['MPP1']
# validation_mpp2 = validation['MPP2']
# validation_ml1 = validation['ML1']
# validation_ml2 = validation['ML2']

In [ ]:
# all_validation_post = pd.concat([validation_post1,validation_post2],axis=0)
# all_validation_mpp = pd.concat([validation_mmp1,validation_mpp2],axis=0)
# all_validation_ml = pd.concat([validation_ml1,validation_ml2],axis=0)

In [18]:
def generate_dataloader(post,mpp_all_value,ml_all_value,max_len,batch_size):
  data = generate_input_data(post,mpp_all_value,ml_all_value,max_len)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader


In [20]:
max_len = 512
batch_size = 1
train_dataloader = generate_dataloader(all_train_post,all_train_mpp,all_train_ml,max_len,batch_size)
# test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,batch_size)
# validation_dataloader = generate_dataloader(all_validation_post,all_validation_mpp,all_validation_ml,max_len,batch_size)

In [21]:
test = df[176:200]

In [24]:

test_post1 = test['post1_en']
test_post2 = test['post2_en']
test_mmp1 = test['MPP1']
test_mpp2 = test['MPP2']
test_ml1 = test['ML1']
test_ml2 = test['ML2']

test_mpp_label = test['MPP_label']
test_ml_label = test['ML_label']

In [25]:
test_dataloader1 = generate_dataloader(test_post1,test_mmp1,test_ml1,max_len,1)
test_dataloader2 = generate_dataloader(test_post2,test_mpp2,test_ml2,max_len,1)

In [28]:
EPOCHS = 100

# total_steps = len(train_data_loader) * EPOCHS

# scheduler = get_linear_schedule_with_warmup(
#   optimizer,
#   num_warmup_steps=0,
#   num_training_steps=total_steps
# )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss().to(device)


label_num = 2
model= BertNerClassifier(label_num).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
mean_squared_error = MeanSquaredError().to(device)

Some weights of the model checkpoint at nlpaueb/sec-bert-shape were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:

def train_and_eval(model, 
  train_loader, 
  # validation_loader,
  loss_fn, 
  optimizer,  
  device,
  epoch,
  label_num
  ):
  model.train()
  accuracy_state = 0
  ml_accuracy_state = 0
  for epo in range(epoch):
    
    global train_loss
    train_loss = []
    
    train_predictions = []
    

    validation_loss = []
    
    validation_predictions = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      targets = data[2].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      

      loss = loss_fn(outputs, targets)
      
      

      train_loss.append(loss.item())
      
      # train_predictions += torch.sum(outputs == targets)
      # print(train_mpp_predictions)
      
      mse = mean_squared_error(outputs, targets)
      train_predictions.append(mse.item())
    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    # model.eval()

    # with torch.no_grad():
    #   for data in validation_loader:
    
    #     input_ids = data[0].to(device)
    #     attention_mask = data[1].to(device)
        
    #     targets = data[2].to(device)
        

    #     outputs = model(
    #       input_ids=input_ids,
    #       attention_mask=attention_mask,
          
    #     )

    #     # preds, _ = torch.max(outputs, dim=1)

    #     loss = loss_fn(outputs, targets)

    
    #     validation_loss.append(loss.item())
        
    #     mse = mean_squared_error(outputs, targets)
    #     validation_predictions.append(mse.item())
        
        # validation_ml_predictions += torch.sum(preds == ml_targets)
    
    train_mse = np.mean(train_predictions)
    
    final_train_loss = np.mean(train_loss)
   

    # validation_mse = np.mean(validation_predictions)
    
    # final_validation_loss = np.mean(validation_loss)
   

    if(train_mse > accuracy_state):
      accuracy_state = train_mse
      torch.save(model.state_dict(), 're_bert_state.bin')

    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train RMSE: {:.4f}'
    .format(epo+1,final_train_loss, train_mse))

In [ ]:
train_and_eval(model, 
  train_dataloader, 
  loss_fn, 
  optimizer,  
  device,
  EPOCHS,
  label_num
  )

Epoch [1] Train Totol Loss: 0.0032, Train RMSE: 0.0032
Epoch [2] Train Totol Loss: 0.0029, Train RMSE: 0.0029
Epoch [3] Train Totol Loss: 0.0025, Train RMSE: 0.0025
Epoch [4] Train Totol Loss: 0.0022, Train RMSE: 0.0022
Epoch [5] Train Totol Loss: 0.0023, Train RMSE: 0.0023
Epoch [6] Train Totol Loss: 0.0023, Train RMSE: 0.0023
Epoch [7] Train Totol Loss: 0.0020, Train RMSE: 0.0020
Epoch [8] Train Totol Loss: 0.0021, Train RMSE: 0.0021
Epoch [9] Train Totol Loss: 0.0020, Train RMSE: 0.0020
Epoch [10] Train Totol Loss: 0.0017, Train RMSE: 0.0017
Epoch [11] Train Totol Loss: 0.0017, Train RMSE: 0.0017
Epoch [12] Train Totol Loss: 0.0019, Train RMSE: 0.0019
Epoch [13] Train Totol Loss: 0.0017, Train RMSE: 0.0017
Epoch [14] Train Totol Loss: 0.0019, Train RMSE: 0.0019
Epoch [15] Train Totol Loss: 0.0017, Train RMSE: 0.0017
Epoch [16] Train Totol Loss: 0.0017, Train RMSE: 0.0017
Epoch [17] Train Totol Loss: 0.0018, Train RMSE: 0.0018
Epoch [18] Train Totol Loss: 0.0016, Train RMSE: 0.0016
E

In [ ]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  ):
  
  
  test_pred = []

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      targets = data[2].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )

      
      test_pred.append(outputs)
  return test_pred
     

  # print('Test ACCURACY_SCORE: {:.4f}'.format(test_accuracy))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_pred1 = test_prediction(model, 
  test_dataloader1, 
  loss_fn, 
  optimizer,
  device
  )

test_pred2 = test_prediction(model, 
  test_dataloader2, 
  loss_fn, 
  optimizer,
  device
  )

mpp_pred_label = []
ml_pred_label = []
for i, j in zip (test_pred1,test_pred2):
  if i[0][0]>j[0][0]:
    mpp_pred_label.append(1)
  else:
    mpp_pred_label.append(0)
  if i[0][1]>j[0][1]:
    ml_pred_label.append(0)
  else:
    ml_pred_label.append(1)


mpp_accuracy = accuracy_score(test_mpp_label, mpp_pred_label)
ml_accuracy = accuracy_score(test_ml_label, ml_pred_label)

In [ ]:
mpp_accuracy

0.5625

In [ ]:
ml_accuracy

0.4375

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/新建文件夹/chinese_bert_state.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>